## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-26-01-09-47 +0000,nypost,Gwyneth Paltrow wades into viral Astronomer sc...,https://nypost.com/2025/07/25/us-news/gwyneth-...
1,2025-07-26-01-02-43 +0000,nypost,NYC Board of Elections asks Brooklyn DA to lau...,https://nypost.com/2025/07/25/us-news/nyc-boar...
2,2025-07-26-01-00-13 +0000,bbc,Salon owner 'ready' for key L'Oréal dispute he...,https://www.bbc.com/news/articles/c0q8y5qn072o
3,2025-07-26-01-00-00 +0000,wsj,The Chinese Coffee Chain That’s Muscling In on...,https://www.wsj.com/business/retail/china-luck...
4,2025-07-26-01-00-00 +0000,wsj,The Hottest Business Strategy This Summer Is B...,https://www.wsj.com/finance/currencies/crypto-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2359/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
132,trump,60
58,new,19
240,gaza,16
81,will,13
430,israel,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
166,2025-07-25-15-18-56 +0000,bbc,Trump says Hamas 'didn't want deal' as US and ...,https://www.bbc.com/news/articles/crrqrz857wgo,119
294,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...,110
149,2025-07-25-16-25-00 +0000,wsj,President Trump has long wanted a U.S. soverei...,https://www.wsj.com/politics/policy/sovereign-...,108
68,2025-07-25-21-29-14 +0000,nypost,Eric Adams insists he can’t nix NYC sanctuary ...,https://nypost.com/2025/07/25/us-news/eric-ada...,104
197,2025-07-25-13-23-00 +0000,wsj,Trump Administration Sues New York City Over S...,https://www.wsj.com/us-news/trump-administrati...,102


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
166,119,2025-07-25-15-18-56 +0000,bbc,Trump says Hamas 'didn't want deal' as US and ...,https://www.bbc.com/news/articles/crrqrz857wgo
84,73,2025-07-25-20-45-00 +0000,wsj,Rep. Alexandria Ocasio-Cortez of New York stil...,https://www.wsj.com/politics/policy/aoc-met-ga...
74,52,2025-07-25-21-10-00 +0000,cbc,France plans to recognize Palestinian statehoo...,https://www.cbc.ca/news/politics/france-macron...
294,50,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...
11,46,2025-07-26-00-25-07 +0000,nyt,Thailand Warns of War With Cambodia as Deadly ...,https://www.nytimes.com/2025/07/25/world/asia/...
37,42,2025-07-25-22-30-20 +0000,nypost,Southwest flight plummets nearly 500 feet ‘in ...,https://nypost.com/2025/07/25/us-news/southwes...
149,36,2025-07-25-16-25-00 +0000,wsj,President Trump has long wanted a U.S. soverei...,https://www.wsj.com/politics/policy/sovereign-...
5,36,2025-07-26-01-00-00 +0000,wsj,The Democratic Party’s image has eroded to its...,https://www.wsj.com/politics/elections/democra...
68,34,2025-07-25-21-29-14 +0000,nypost,Eric Adams insists he can’t nix NYC sanctuary ...,https://nypost.com/2025/07/25/us-news/eric-ada...
192,30,2025-07-25-13-47-06 +0000,nypost,Trump says ‘I’ll give you a list’ of Jeffrey E...,https://nypost.com/2025/07/25/us-news/trump-sa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
